# Install Required Packages and Loading Data

In [1]:
import os
import openai
import sys
sys.path.append('../..')

In [2]:
from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv())

In [3]:
from dotenv import load_dotenv
load_dotenv('.env')
api_key = os.getenv('OPENAI_API_KEY')

## Import a PDF File

In [4]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("MachineLearning-Lecture01.pdf")
pages = loader.load()

In [5]:
len(pages)

22

In [6]:
page = pages[2]

In [7]:
print(page.page_content[0:500])

So in this class, we've tried to convey to you a broad set of principl es and tools that will 
be useful for doing many, many things. And ev ery time I teach this class, I can actually 
very confidently say that af ter December, no matter what yo u're going to do after this 
December when you've sort of completed this  class, you'll find the things you learn in 
this class very useful, and these things will be useful pretty much no matter what you end 
up doing later in your life.  
So I have mo


In [8]:
page.metadata

{'source': 'MachineLearning-Lecture01.pdf', 'page': 2}

## Import a Youtube Video
   

In [9]:
'''from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader'''

'from langchain.document_loaders.generic import GenericLoader\nfrom langchain.document_loaders.parsers import OpenAIWhisperParser\nfrom langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader'

In [10]:
#import ffmpeg

In [11]:
'''url="https://www.youtube.com/watch?v=jGwO_UgTS7I"
save_dir="docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url],save_dir),
    OpenAIWhisperParser()
)
docs = loader.load()'''

'url="https://www.youtube.com/watch?v=jGwO_UgTS7I"\nsave_dir="docs/youtube/"\nloader = GenericLoader(\n    YoutubeAudioLoader([url],save_dir),\n    OpenAIWhisperParser()\n)\ndocs = loader.load()'

In [12]:
#docs[0].page_content[0:500]

## Import a URL

In [13]:
#from langchain.document_loaders import WebBaseLoader

#loader = WebBaseLoader("https://github.com/basecamp/handbook/blob/master/37signals-is-you.md")

In [14]:
#docs = loader.load()

In [15]:
#print(docs[0].page_content[:500])

# Document Splitting 

In [16]:
from langchain.text_splitter import CharacterTextSplitter

In [17]:
text_splitter = CharacterTextSplitter(
     separator="\n",
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len
)


In [18]:
docs = text_splitter.split_documents(pages)

In [19]:
splits = text_splitter.split_documents(docs)

In [20]:
len(docs),len(pages),len(splits)

(57, 22, 57)

## Token Splitting

In [21]:
from langchain.text_splitter import TokenTextSplitter

In [22]:
text_splitter = TokenTextSplitter(chunk_size = 10,chunk_overlap= 0)

In [23]:
docs = text_splitter.split_documents(pages)

In [24]:
len(docs)

1557

In [25]:
docs[1000]

Document(metadata={'source': 'MachineLearning-Lecture01.pdf', 'page': 13}, page_content=' call support vector machines — actually takes \ndata')

In [26]:
pages[0].metadata

{'source': 'MachineLearning-Lecture01.pdf', 'page': 0}

## Vectorstore and Embeddigns

In [27]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

C:\Users\purva\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [28]:
from langchain.vectorstores import Chroma

In [29]:
persist_directory = 'docs/chroma/'

In [30]:
!del -rf ./docs/chroma

Invalid switch - "docs".


In [31]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [32]:
print(vectordb._collection.count())

342


## Similarity Search

In [33]:
question = "is there an email that I can get"

In [34]:
docs = vectordb.similarity_search(question,k=3)

In [35]:
len(docs)

3

In [36]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an acc ount that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework probl ems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me  appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thi ng that I think will help you to succeed and \ndo well in this class and even help you to enjoy this cla ss more is if you form a study \ngroup.  \nSo start looking around where you' re sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to f

In [37]:
vectordb.persist()

C:\Users\purva\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


## Failure Mode

In [38]:
question = "what did they say about cats?"

In [39]:
docs = vectordb.similarity_search(question,k=5)

In [40]:
docs[0]

Document(metadata={'page': 4, 'source': 'MachineLearning-Lecture01.pdf'}, page_content="middle of class, but because there won't be video you can safely sit there and make faces \nat me, and that won't show, okay?  \nLet's see. I also handed out this — ther e were two handouts I hope most of you have, \ncourse information handout. So let me just sa y a few words about parts of these. On the \nthird page, there's a section that says Online Resources.  \nOh, okay. Louder? Actually, could you turn up the volume? Testing. Is this better? \nTesting, testing. Okay, cool. Thanks.")

In [41]:
docs[1]

Document(metadata={'page': 4, 'source': 'MachineLearning-Lecture01.pdf'}, page_content="middle of class, but because there won't be video you can safely sit there and make faces \nat me, and that won't show, okay?  \nLet's see. I also handed out this — ther e were two handouts I hope most of you have, \ncourse information handout. So let me just sa y a few words about parts of these. On the \nthird page, there's a section that says Online Resources.  \nOh, okay. Louder? Actually, could you turn up the volume? Testing. Is this better? \nTesting, testing. Okay, cool. Thanks.")

In [42]:
question = "what did they say about regression in the lecture?"

In [43]:
docs = vectordb.similarity_search(question,k=5)

In [44]:
for doc in docs:
    print(doc.metadata)

{'page': 8, 'source': 'MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'MachineLearning-Lecture01.pdf'}
{'page': 8, 'source': 'MachineLearning-Lecture01.pdf'}


In [45]:
print(docs[4].page_content)

into his office and he said, "Oh, professo r, professor, thank you so much for your 
machine learning class. I learned so much from it. There's this stuff that I learned in your 
class, and I now use every day. And it's help ed me make lots of money, and here's a 
picture of my big house."  
So my friend was very excited. He said, "W ow. That's great. I'm glad to hear this 
machine learning stuff was actually useful. So what was it that you learned? Was it 
logistic regression? Was it the PCA? Was it the data ne tworks? What was it that you 
learned that was so helpful?" And the student said, "Oh, it was the MATLAB."  
So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, 
and we'll actually have a short MATLAB tutori al in one of the discussion sections for 
those of you that don't know it.  
Okay. The very last piece of logistical th ing is the discussion s ections. So discussion 
sections will be taught by the TAs, and atte ndance at discussion secti

## Maximum Marginal Relevance

In [46]:
## MMR allows for diverse results

In [47]:
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)

In [48]:
docs_ss[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [49]:
docs_ss[1].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [50]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [51]:
docs_mmr[0].page_content[:100]

'those homeworks will be done in either MATLA B or in Octave, which is sort of — I \nknow some people '

In [52]:
docs_mmr[1].page_content[:100]

'into his office and he said, "Oh, professo r, professor, thank you so much for your \nmachine learnin'

## Self Query

In [53]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [54]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [55]:
#!pip install lark

In [56]:
document_content_description = "Lecture notes"
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

C:\Users\purva\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [57]:
question = "what ia said about Amazon on page 4?"

In [58]:
docs = retriever.get_relevant_documents(question)

C:\Users\purva\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [59]:
for d in docs:
    print(d.metadata)

{'page': 4, 'source': 'MachineLearning-Lecture01.pdf'}
{'page': 4, 'source': 'MachineLearning-Lecture01.pdf'}
{'page': 4, 'source': 'MachineLearning-Lecture01.pdf'}
{'page': 4, 'source': 'MachineLearning-Lecture01.pdf'}


## Compression 

In [60]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [61]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [62]:
# Wrap our vectorstore
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
compressor = LLMChainExtractor.from_llm(llm)

In [63]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [64]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

- "those homeworks will be done in either MATLA B or in Octave"
- "I know some people call it a free ve rsion of MATLAB"
- "MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data."
- "there's also a software package called Octave that you can download for free off the Internet."
- "it has somewhat fewer features than MATLAB, but it's free, and for the purposes of this class, it will work for just about everything."
- "once a colleague of mine at a different university, not at Stanford, actually teaches another machine learning course."
----------------------------------------------------------------------------------------------------
Document 2:

- "those homeworks will be done in either MATLA B or in Octave"
- "I know some people call it a free ve rsion of MATLAB"
- "MATLAB is I guess part of the programming language that makes it very easy to write 

## Combining Techniques

In [65]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type = "mmr")
)

In [66]:
question = "what did they say about matlab?"
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

- "those homeworks will be done in either MATLA B or in Octave"
- "I know some people call it a free ve rsion of MATLAB"
- "MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to plot data."
- "there's also a software package called Octave that you can download for free off the Internet."
- "it has somewhat fewer features than MATLAB, but it's free, and for the purposes of this class, it will work for just about everything."
- "once a colleague of mine at a different university, not at Stanford, actually teaches another machine learning course."
----------------------------------------------------------------------------------------------------
Document 2:

"Oh, it was the MATLAB."
----------------------------------------------------------------------------------------------------
Document 3:

- All the homeworks can be done in MATLAB or Octave
- The program pre

## Question Answering

In [67]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [68]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

C:\Users\purva\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [69]:
print(vectordb._collection.count())

342


In [70]:
question = "What are major topics for this class?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

In [71]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

## Retrieval Q&A Chain

In [72]:
from langchain.chains import RetrievalQA

In [73]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [74]:
result = qa_chain({"query": question})

C:\Users\purva\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


In [75]:
result["result"]

'The major topics for this class include machine learning, statistics, and algebra. Additionally, there will be discussions on extensions of the material covered in the main lectures.'

## Prompt

In [76]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [77]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

In [78]:
question = "Is cooking a class topic?"

In [79]:
result = qa_chain({"query": question})

In [80]:
result["result"]

'No, cooking is not a class topic. Thanks for asking!'

In [81]:
result["source_documents"][0]

Document(metadata={'page': 4, 'source': 'MachineLearning-Lecture01.pdf'}, page_content="But if you don't quite know or if you're not qu ite sure, that's fine, too. We'll go over it in \nthe review sections.  \nSo there are a couple more logisti cal things I should deal with in  this class. One is that, as \nmost of you know, CS229 is a televised cla ss. And in fact, I guess many of you are \nprobably watching this at home on TV, so I' m gonna say hi to our home viewers.  \nSo earlier this year, I appro ached SCPD, which televises th ese classes, about trying to \nmake a small number of Stanford classes publ icly available or posting the videos on the \nweb. And so this year, Stanford is actually starting a small pilot program in which we'll \npost videos of a small number of classes onlin e, so on the Internet in a way that makes it \npublicly accessible to everyone. I'm very exc ited about that because machine learning in \nschool, let's get the word out there.  \nOne of the consequen

In [82]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [83]:
result = qa_chain_mr({"query": question})

In [84]:
result["result"]

'No, cooking is not mentioned as a class topic in the provided text. The text specifically discusses a televised class on machine learning at Stanford University and the logistics related to posting videos of classes online.'

In [85]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
result = qa_chain_mr({"query": question})
result["result"]

'No, cooking is not mentioned as a class topic in the provided text.'

In [86]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

"Based on the additional context provided, cooking is not mentioned as a class topic in CS229. The class focuses on machine learning concepts and is part of a televised program where videos are posted online for public access. The instructor mentions that student images will not be posted online, but their voices may be picked up by microphones. Therefore, it is important for students to refrain from yelling out swear words during class. The instructor also mentions that the class is part of a pilot program where a small number of Stanford classes will be publicly available online, which includes CS229. The instructor also emphasizes that students can safely make faces at him during class as there won't be video footage of them."

## Limitations

In [87]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [88]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})
result["result"]

'Yes, probability is a class topic for the course being described. The instructor assumes familiarity with basic probability and statistics.'

In [89]:
question = "why are those prerequesites needed?"
result = qa_chain({"query": question})
result["result"]

'The prerequisites are needed to ensure that all students have a basic understanding of computer science concepts and skills before delving into more advanced topics in machine learning. This foundational knowledge is essential for comprehending and applying the state-of-the-art machine learning algorithms taught in the class.'

## Memory

In [90]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## Conversational Retrival Chain

In [91]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [92]:
question = "Is probability a class topic?"
result = qa({"question": question})

In [93]:
result['answer']

'Yes, probability is a class topic for the course being described. The instructor assumes familiarity with basic probability and statistics.'

In [94]:
question = "why are those prerequesites needed?"
result = qa({"question": question})

In [95]:
result['answer']

'Familiarity with basic probability and statistics is assumed as prerequisites for the course because the course content will involve concepts and applications that build upon these foundational principles. Understanding random variables, expectations, variances, and other statistical concepts will be essential for grasping the material covered in the course.'

## User Interface

In [110]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [111]:
def load_db(file, chain_type, k):
    # load documents
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    # define embedding
    embeddings = OpenAIEmbeddings()
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa 


In [116]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "MachineLearning-Lecture01.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)
    
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 


In [118]:
#pn.extension()

In [119]:
## Creating chatbot

cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()
        [3] Column
            [0] Row
                [0] FileInput(accept='.pdf')
                [1] Button(button_type='primary', name='Load DB')
                [2] ParamFunction(function, _pane=Markdown, defer_load=False)
            [1] Row
                [0] Button(button_type='warning', name='Clear History')
                [1] Markdown(str)
            [2] Divider()
            [3] Row
                [0] Image(str, width=400)